<a href="https://colab.research.google.com/github/dustinak80/Deep_Learning/blob/master/HW3/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Import packages

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import Constant

## Import the Data

In [5]:
root = 'drive/My Drive/Deep learning/'

stocks = pd.read_csv(root+'stocks_dataset.csv')
stocks.head()

,Date,ABT,ABBV,ABMD,A,ALXN,ALGN,AGN,ABC,AMGN,ANTM,BAX,BDX,BIIB,BSX,BMY,CAH,CNC,CERN,CI,COO,CVS,DHR,DVA,XRAY,EW,GILD,HCA,HSIC,HOLX,HUM,IDXX,ILMN,INCY,ISRG,IQV,JNJ,LH,LLY,MCK,MDT,MRK,MTD,MYL,PKI,PRGO,PFE,DGX,REGN,RMD,STE
0,2015-01-02 00:00:00,44.900002,65.889999,37.310001,40.560001,186.600006,56.200001,259.130005,90.459999,159.889999,125.459999,39.820751,139.860001,342.010010,13.22,59.509998,80.510002,26.110001,65.029999,102.830002,160.610001,95.099998,64.988625,75.830002,51.930000,63.860001,94.910004,73.970001,53.662746,26.379999,142.990005,73.665001,185.419998,73.760002,175.190002,59.110001,104.519997,111.419998,70.059998,207.199997,71.879997,57.189999,301.950012,56.349998,44.090000,166.630005,31.33,68.970001,410.160004,57.169998,65.160004
1,2015-01-05 00:00:00,44.910000,64.650002,37.070000,39.799999,182.169998,56.990002,256.690002,89.690002,157.990005,123.260002,38.832157,140.720001,336.739990,13.81,58.580002,80.050003,26.040001,64.169998,101.059998,159.369995,94.160004,64.344200,74.699997,51.570000,63.889999,96.790001,71.809998,53.180393,26.120001,139.199997,73.154999,190.720001,74.220001,171.456665,58.410000,103.790001,112.389999,69.370003,208.899994,71.580002,58.040001,295.450012,55.720001,43.049999,164.910004,31.16,67.279999,412.470001,57.060001,64.510002
2,2015-01-06 00:00:00,44.400002,64.330002,36.130001,39.180000,177.949997,57.450001,255.169998,90.180000,152.899994,124.940002,38.625748,140.449997,334.649994,13.70,58.480000,79.190002,26.125000,63.520000,101.459999,157.119995,94.230003,63.904472,73.620003,50.930000,63.509998,97.650002,71.690002,52.870586,25.750000,139.089996,73.389999,188.309998,71.489998,173.263336,57.070000,103.279999,110.599999,69.720001,208.419998,71.220001,60.320000,291.119995,54.180000,42.660000,164.600006,31.42,66.269997,396.890015,56.650002,64.550003
3,2015-01-07 00:00:00,44.759998,66.930000,37.279999,39.700001,187.929993,59.570000,265.230011,91.980003,158.240005,125.559998,38.810429,143.740005,353.239990,14.03,59.240002,82.029999,26.267500,64.190002,103.839996,162.220001,95.930000,64.291130,74.290001,52.250000,65.000000,99.480003,72.989998,54.333332,26.400000,141.729996,74.870003,194.919998,74.449997,174.213333,57.770000,105.559998,112.540001,69.230003,214.750000,73.000000,61.610001,296.489990,56.110001,42.919998,167.350006,31.85,67.110001,407.720001,57.029999,65.070000
4,2015-01-08 00:00:00,45.680000,67.629997,38.959999,40.889999,183.800003,61.439999,268.959992,92.190002,157.669998,131.550003,39.815319,146.360001,350.250000,14.59,60.720001,84.559998,26.672501,65.599998,109.129997,162.880005,97.970001,65.284309,75.769997,53.540001,66.574997,102.300003,74.930000,55.098038,27.110001,147.830002,75.415001,196.820007,73.209999,177.666672,59.200001,106.389999,115.570000,70.870003,217.199997,74.750000,62.849998,300.720001,56.650002,44.000000,171.169998,32.50,68.879997,403.250000,57.990002,66.779999


In [6]:
stocks.tail()

,Date,ABT,ABBV,ABMD,A,ALXN,ALGN,AGN,ABC,AMGN,ANTM,BAX,BDX,BIIB,BSX,BMY,CAH,CNC,CERN,CI,COO,CVS,DHR,DVA,XRAY,EW,GILD,HCA,HSIC,HOLX,HUM,IDXX,ILMN,INCY,ISRG,IQV,JNJ,LH,LLY,MCK,MDT,MRK,MTD,MYL,PKI,PRGO,PFE,DGX,REGN,RMD,STE
1332,2020-04-20 00:00:00,98.000000,83.989998,166.199997,77.370003,106.720001,196.710007,187.389999,89.620003,236.600006,263.200012,94.139999,263.440002,339.410004,37.250000,61.680000,50.740002,68.419998,70.250000,188.699997,304.700012,62.340000,155.830002,77.519997,41.279999,224.600006,81.260002,110.430000,53.150002,43.240002,367.329987,267.339996,322.880005,101.190002,518.330017,127.510002,151.669998,145.610001,157.789993,140.490005,101.540001,83.099998,707.630005,15.86,84.519997,50.860001,36.080002,95.029999,567.989990,164.279999,157.449997
1333,2020-04-21 00:00:00,94.050003,80.360001,164.500000,73.750000,103.389999,187.259995,184.039993,85.570000,230.750000,249.369995,90.099998,256.500000,329.010010,35.049999,60.590000,48.270000,66.239998,68.589996,177.910004,288.529999,60.250000,153.750000,72.529999,39.660000,216.020004,78.529999,105.470001,52.029999,42.349998,358.859985,259.230011,310.399994,99.459999,496.630005,122.000000,149.679993,148.179993,152.669998,132.440002,97.610001,78.559998,683.429993,15.13,82.639999,49.689999,35.619999,94.459999,544.460022,157.350006,147.779999
1334,2020-04-22 00:00:00,95.480003,81.470001,166.729996,75.169998,103.910004,190.029999,185.339996,86.500000,229.289993,262.959992,90.540001,266.760010,298.010010,35.490002,60.930000,49.040001,68.349998,70.779999,182.589996,294.100006,60.349998,160.250000,72.410004,39.910000,216.520004,81.309998,104.519997,51.709999,42.880001,367.170013,268.119995,311.559998,100.970001,512.130005,125.550003,152.990005,154.130005,156.710007,135.220001,96.080002,79.980003,704.200012,14.91,85.550003,49.619999,36.250000,99.580002,558.080017,160.919998,150.880005
1335,2020-04-23 00:00:00,93.940002,82.040001,170.759995,75.089996,104.139999,188.570007,185.550003,87.260002,232.490006,261.549988,91.320000,265.079987,292.250000,36.400002,60.950001,49.320000,69.239998,71.150002,184.279999,290.269989,61.279999,160.649994,73.349998,38.700001,221.039993,77.779999,104.739998,51.630001,45.380001,366.070007,268.410004,312.709992,101.900002,513.219971,127.669998,155.509995,159.669998,159.929993,136.570007,95.830002,80.879997,703.299988,15.30,87.199997,50.990002,36.689999,100.660004,565.520019,158.729996,149.449997
1336,2020-04-24 00:00:00,94.059998,83.589996,168.520004,75.750000,108.120003,191.970001,186.899994,88.699997,236.279999,264.119995,92.919998,268.170013,297.799988,36.900002,62.250000,50.320000,70.580002,72.750000,188.199997,293.209992,62.310001,165.070007,74.000000,39.310001,223.479996,79.639999,107.830002,52.389999,47.110001,369.089996,271.799988,309.070007,103.480003,514.929993,128.860001,154.860001,163.720001,162.929993,139.940002,99.300003,81.430000,713.380005,15.76,87.550003,51.430000,37.380001,106.620003,566.210022,161.000000,147.820007


In [7]:
#any nulls?
print('There are {} null values\n'.format(stocks.isnull().sum().sum()))

There are 0 null values



In [8]:
#what are the dtypes of the columns
stocks.dtypes

Date     object
ABT     float64
ABBV    float64
ABMD    float64
A       float64
ALXN    float64
ALGN    float64
AGN     float64
ABC     float64
AMGN    float64
ANTM    float64
BAX     float64
BDX     float64
BIIB    float64
BSX     float64
BMY     float64
CAH     float64
CNC     float64
CERN    float64
CI      float64
COO     float64
CVS     float64
DHR     float64
DVA     float64
XRAY    float64
EW      float64
GILD    float64
HCA     float64
HSIC    float64
HOLX    float64
HUM     float64
IDXX    float64
ILMN    float64
INCY    float64
ISRG    float64
IQV     float64
JNJ     float64
LH      float64
LLY     float64
MCK     float64
MDT     float64
MRK     float64
MTD     float64
MYL     float64
PKI     float64
PRGO    float64
PFE     float64
DGX     float64
REGN    float64
RMD     float64
STE     float64
dtype: object

Dates are not in date format. Lets get into date format.

In [9]:
stocks['Date'] = pd.to_datetime(stocks['Date'])
stocks['Date'].dtypes

dtype('<M8[ns]')

In [10]:
print('The shape is {}'.format(stocks.shape))

The shape is (1337, 51)


#### Normalize

In [0]:
stocks2 = stocks.copy(deep = True)

In [0]:
means = stocks2.loc[:,'ABT':].mean()
stocks2.loc[:,'ABT':] = stocks2.loc[:,'ABT':]/means

In [25]:
stocks2.describe()

,ABT,ABBV,ABMD,A,ALXN,ALGN,AGN,ABC,AMGN,ANTM,BAX,BDX,BIIB,BSX,BMY,CAH,CNC,CERN,CI,COO,CVS,DHR,DVA,XRAY,EW,GILD,HCA,HSIC,HOLX,HUM,IDXX,ILMN,INCY,ISRG,IQV,JNJ,LH,LLY,MCK,MDT,MRK,MTD,MYL,PKI,PRGO,PFE,DGX,REGN,RMD,STE
count,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000
mean,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,0.281322,0.205326,0.572711,0.257137,0.193552,0.573263,0.253446,0.108332,0.126506,0.296051,0.291251,0.213370,0.156027,0.311692,0.122885,0.223189,0.300780,0.106877,0.174486,0.262575,0.199054,0.288749,0.128500,0.147636,0.389570,0.189526,0.260004,0.093054,0.133372,0.251532,0.445291,0.293225,0.222724,0.432389,0.309944,0.125502,0.153227,0.218082,0.198559,0.130949,0.168074,0.302842,0.350064,0.249263,0.470215,0.105204,0.162301,0.177741,0.354138,0.317990
min,0.629754,0.641856,0.198769,0.572905,0.567693,0.304580,0.537768,0.775620,0.749548,0.577763,0.542530,0.650664,0.721034,0.470398,0.727759,0.609518,0.561962,0.749533,0.624546,0.537815,0.651675,0.647421,0.639256,0.586726,0.479017,0.758302,0.643207,0.697124,0.633411,0.577812,0.382248,0.533132,0.638071,0.431145,0.564097,0.735254,0.691701,0.713121,0.677325,0.750601,0.738372,0.527990,0.343698,0.606176,0.391003,0.792920,0.668345,0.676404,0.557783,0.651990
25%,0.762152,0.830942,0.551746,0.733086,0.876559,0.434251,0.782697,0.929216,0.902813,0.711138,0.728474,0.805596,0.902798,0.733351,0.904551,0.779588,0.729508,0.916781,0.849497,0.783856,0.839440,0.767449,0.905882,0.931220,0.692498,0.846858,0.791033,0.922521,0.921951,0.754035,0.524388,0.754957,0.828838,0.592613,0.732690,0.908107,0.858767,0.845865,0.863757,0.901955,0.873786,0.687004,0.789780,0.765580,0.637175,0.928181,0.841423,0.892811,0.699965,0.727544
50%,0.883458,0.937586,0.813673,1.059970,0.947032,1.003748,0.937016,0.980677,0.981373,0.942943,1.043871,0.992113,0.977826,0.960010,1.009198,1.036626,0.952496,1.012193,0.987495,1.013320,0.976825,0.901506,1.004965,1.039001,0.892597,0.933536,0.881838,1.001535,0.975330,0.995686,0.984121,0.916050,0.937473,0.938934,0.949208,1.029582,0.982593,0.904995,0.936164,0.971407,0.952929,1.081352,1.000250,0.997083,0.866207,0.975773,1.025657,0.969045,0.873292,0.904747
75%,1.237673,1.174542,1.435508,1.199204,1.073103,1.489101,1.172513,1.046408,1.076823,1.284169,1.247467,1.205126,1.079272,1.288434,1.086789,1.207291,1.238578,1.081181,1.144713,1.189451,1.203967,1.109779,1.100809,1.114227,1.245691,1.092361,1.272491,1.077320,1.085285,1.196294,1.380435,1.298750,1.165215,1.444713,1.260667,1.098626,1.141711,1.191562,1.114147,1.054006,1.137599,1.229334,1.175209,1.223027,1.063315,1.038951,1.140527,1.111914,1.213852,1.211065
max,1.698292,1.638349,2.474302,1.547377,1.561532,2.282053,1.577569,1.297532,1.378479,1.564524,1.582703,1.427020,1.583673,1.626466,1.306291,1.360265,1.628669,1.269474,1.398030,1.599890,1.418234,1.763069,1.302951,1.261368,1.904774,1.530758,1.546236,1.190018,1.344307,1.579439,1.795071,1.652569,1.665092,1.758162,1.675946,1.260215,1.338174,1.761237,1.519178,1.411158,1.403547,1.659266,1.971469,1.487727,2.182014,1.286651,1.277709,1.465304,1.993033,1.773190


In [26]:
stocks2.head()

,Date,ABT,ABBV,ABMD,A,ALXN,ALGN,AGN,ABC,AMGN,ANTM,BAX,BDX,BIIB,BSX,BMY,CAH,CNC,CERN,CI,COO,CVS,DHR,DVA,XRAY,EW,GILD,HCA,HSIC,HOLX,HUM,IDXX,ILMN,INCY,ISRG,IQV,JNJ,LH,LLY,MCK,MDT,MRK,MTD,MYL,PKI,PRGO,PFE,DGX,REGN,RMD,STE
0,2015-01-02,0.778095,0.876153,0.205261,0.696345,1.401953,0.326356,1.204110,1.016408,0.906271,0.618377,0.669475,0.697867,1.137930,0.470398,1.012601,1.196884,0.579038,1.028966,0.635485,0.713813,1.188841,0.677784,1.116415,0.955130,0.493945,1.188808,0.757250,0.866364,0.648908,0.594014,0.454459,0.810140,0.804515,0.498168,0.596383,0.847004,0.763516,0.757333,1.296699,0.836225,0.872108,0.574262,1.460587,0.644784,1.785011,0.871962,0.761787,1.014532,0.645123,0.685663
1,2015-01-05,0.778268,0.859664,0.203941,0.683297,1.368670,0.330944,1.192772,1.007756,0.895501,0.607533,0.652855,0.702159,1.120396,0.491391,0.996776,1.190046,0.577486,1.015358,0.624546,0.708302,1.177091,0.671063,1.099779,0.948509,0.494177,1.212356,0.735138,0.858576,0.642512,0.578269,0.451313,0.833297,0.809532,0.487552,0.589320,0.841089,0.770163,0.749874,1.307338,0.832735,0.885070,0.561900,1.444258,0.629575,1.766586,0.867230,0.743121,1.020246,0.643882,0.678823
2,2015-01-06,0.769430,0.855409,0.198769,0.672653,1.336964,0.333615,1.185709,1.013261,0.866651,0.615814,0.649385,0.700811,1.113442,0.487477,0.995075,1.177261,0.579371,1.005073,0.627018,0.698302,1.177966,0.666477,1.083878,0.936738,0.491238,1.223128,0.733909,0.853574,0.633411,0.577812,0.452762,0.822767,0.779755,0.492690,0.575800,0.836956,0.757897,0.753658,1.304334,0.828546,0.919838,0.553665,1.404341,0.623871,1.763265,0.874467,0.731965,0.981709,0.639256,0.679244
3,2015-01-07,0.775669,0.889982,0.205096,0.681581,1.411945,0.345926,1.232456,1.033486,0.896918,0.618870,0.652490,0.717228,1.175294,0.499219,1.008007,1.219481,0.582531,1.015674,0.641727,0.720968,1.199217,0.670510,1.093743,0.961016,0.502763,1.246050,0.747218,0.877190,0.649400,0.588779,0.461893,0.851648,0.812040,0.495391,0.582863,0.855432,0.771191,0.748361,1.343948,0.849254,0.939510,0.563878,1.454367,0.627674,1.792724,0.886434,0.741243,1.008497,0.643544,0.684716
4,2015-01-08,0.791612,0.899290,0.214339,0.702011,1.380916,0.356785,1.249788,1.035846,0.893687,0.648394,0.669384,0.730301,1.165346,0.519145,1.033190,1.257093,0.591513,1.037985,0.674419,0.723902,1.224719,0.680868,1.115532,0.984743,0.514945,1.281373,0.767078,0.889536,0.666865,0.614120,0.465255,0.859949,0.798516,0.505211,0.597291,0.862158,0.791954,0.766089,1.359281,0.869613,0.958419,0.571923,1.468363,0.643468,1.833645,0.904525,0.760793,0.997441,0.654377,0.702710


#### Series to Supervised

In [0]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  """
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""#n_vars = 1 if type(data) is list else data.shape[1]
  variables = list(data.columns)
  df = data.copy(deep = True)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += ['{}(t-{})'.format(j, i) for j in variables]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
      names += ['{}(t)'.format(j) for j in variables]
    else:
      names += ['{}(t+{})'.format(j, i) for j in variables]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(inplace=True)
  return agg

In [0]:
df = series_to_supervised(stocks2.loc[:,'ABT':], n_in=4, n_out=2, dropnan=False)

In [39]:
print('new shape is {}'.format(df.shape))
#shape is 350 because it is all 50 stocks of t+1. Remove all except ABT
columns = list(df.columns)[-49:]

new shape is (1337, 300)


In [0]:
df.drop(columns = list(df.columns)[-49:], inplace = True)

In [43]:
print('new shape is {}.'.format(df.shape))

new shape is (1337, 251).


In [41]:
#Look at the y values
df.iloc[0:5,[0, 50, 100, 150, 200, 250]]

,ABT(t-4),ABT(t-3),ABT(t-2),ABT(t-1),ABT(t),ABT(t+1)
0,NaN,NaN,NaN,NaN,0.778095,0.778268
1,NaN,NaN,NaN,0.778095,0.778268,0.769430
2,NaN,NaN,0.778095,0.778268,0.769430,0.775669
3,NaN,0.778095,0.778268,0.769430,0.775669,0.791612
4,0.778095,0.778268,0.769430,0.775669,0.791612,0.783294


In [42]:
df.iloc[-5:, [0, 50, 100, 150, 200, 250]]

,ABT(t-4),ABT(t-3),ABT(t-2),ABT(t-1),ABT(t),ABT(t+1)
1332,1.544752,1.575945,1.663633,1.663806,1.698292,1.629840
1333,1.575945,1.663633,1.663806,1.698292,1.629840,1.654621
1334,1.663633,1.663806,1.698292,1.629840,1.654621,1.627934
1335,1.663806,1.698292,1.629840,1.654621,1.627934,1.630013
1336,1.698292,1.629840,1.654621,1.627934,1.630013,NaN


In [46]:
#Save the last day as a forecast of 1 day ahead of last day
last_day = df.iloc[1336,0:250]
last_day

ABT(t-4)     1.698292
ABBV(t-4)    1.116832
ABMD(t-4)    0.914350
A(t-4)       1.328310
ALXN(t-4)    0.801803
               ...   
PFE(t)       1.040342
DGX(t)       1.177639
REGN(t)      1.400523
RMD(t)       1.816772
STE(t)       1.555475
Name: 1336, Length: 250, dtype: float64

Since we are shifting the data, we will have to remove the rows with Nan's in them. Should reduce the rows by 5.

In [37]:
df.dropna(inplace = True)
df.shape

(1332, 251)